In [1]:
# ===============================
# PyTorch Geometric による GCN 学習準備
# ===============================

# 時間計測用モジュール（学習時間の測定などに使用）
import time

# PyTorch本体のインポート（テンソル操作、モデル構築などに使用）
import torch

# PyTorch の関数型API（活性化関数、損失関数などを利用）
import torch.nn.functional as F

# GCNConv モジュールのインポート（Graph Convolutional Network の1層を構成）
# - メッセージパッシングベースのグラフ畳み込み演算
from torch_geometric.nn import GCNConv

# Redditデータセットの読み込み用モジュール
# - ノード分類のベンチマークタスクとして有名
from torch_geometric.datasets import Reddit

# 近傍サンプリングを使ったデータローダ
# - 大規模グラフを扱う際に、各ミニバッチで近傍ノードをサンプリングして効率化
from torch_geometric.loader import NeighborLoader

In [2]:
# ===============================
# Redditデータセットの読み込みと平均次数の計算
# ===============================

# Reddit データセットをローカルの /tmp/Reddit ディレクトリにダウンロード＆キャッシュ
# - 自動的に前処理済みの PyG 形式データに変換される
# - dataset[0] はこのデータセット内の唯一のグラフデータ（大規模な単一グラフ）
dataset = Reddit(root="/tmp/Reddit")

# dataset[0] で得られるのは 1 つの Data オブジェクト
# - data.x: ノード特徴行列（[num_nodes, num_node_features]）
# - data.edge_index: エッジ情報（[2, num_edges] のインデックス形式）
# - data.y: ノードのラベル（整数）
# - その他、train_mask, val_mask, test_mask 等のマスクも含まれる
data = dataset[0]

# グラフの平均次数を表示
# - data.num_edges：エッジ数（片方向で数える）
# - data.num_nodes：ノード数
# - 無向グラフでは、全エッジ数 = ノードの次数の合計 / 2
#   → よって、平均次数は 2 * (エッジ数 / ノード数)
print(2 * data.num_edges / data.num_nodes)  # 平均次数の出力

983.9752065760952


In [3]:
# ===============================
# GCN（Graph Convolutional Network）モデルの定義
# - 2層のGCNConvを用いたシンプルなノード分類モデル
# ===============================


class GCN(torch.nn.Module):
    def __init__(self, in_d, mid_d, out_d):
        super().__init__()

        # 第1層：入力次元 → 中間次元 への GCNConv（グラフ畳み込み層）
        # - 各ノードの特徴ベクトルを近傍ノードと融合しながら変換
        self.conv1 = GCNConv(in_d, mid_d)

        # 第2層：中間次元 → 出力次元（クラス数）への GCNConv
        self.conv2 = GCNConv(mid_d, out_d)

    def forward(self, data):
        # ノード特徴行列とエッジ情報を抽出
        # - data.x: ノード特徴（[num_nodes, in_d]）
        # - data.edge_index: 隣接ノードのペア情報（[2, num_edges]）
        x, edge_index = data.x, data.edge_index

        # 第1層のGCN演算：各ノードが近傍ノードの情報を集約しつつ次元変換
        x = self.conv1(x, edge_index)

        # 活性化関数ReLUを適用（非線形変換）
        x = F.relu(x)

        # 第2層のGCN演算（出力次元への変換）
        x = self.conv2(x, edge_index)

        # 出力を softmax（対数形式）に変換して返す
        # - 各ノードに対するクラス分類の対数確率ベクトル（[num_nodes, out_d]）
        return F.log_softmax(x, dim=1)

In [4]:
# ===============================
# テストデータに対するモデルの分類精度（accuracy）を計算する関数
# ===============================
def calc_acc(model):
    # モデルを推論モードに切り替え（ドロップアウトやBatchNormを無効化）
    model.eval()

    # モデルにデータを入力し、出力のクラスごとのスコアを取得
    # - 出力形状: [num_nodes, num_classes]
    # - argmax(dim=1)：各ノードについて最大スコアのクラスを予測ラベルとして選ぶ
    pred = model(data).argmax(dim=1)

    # テストノード（data.test_mask が True）に限定して、正解と一致した数をカウント
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()

    # テストデータの総数で割って、分類精度（accuracy）を計算
    acc = int(correct) / int(data.test_mask.sum())

    # 精度（0〜1の浮動小数）を返す
    return acc

In [5]:
# ===============================
# モデルと最適化手法（Optimizer）の初期化
# ===============================

# GCN モデルのインスタンスを生成
# - 入力次元：dataset.num_node_features → ノード特徴の次元数（Redditでは 602）
# - 中間層の次元数：32（隠れ層のユニット数）
# - 出力次元：dataset.num_classes → 分類クラス数（Redditでは 41）
model = GCN(dataset.num_node_features, 32, dataset.num_classes)

# 最適化手法として Adam Optimizer を使用
# - 学習率 lr=0.1：勾配更新のステップサイズ（やや大きめ）
# - weight_decay=1e-4：L2正則化項。過学習を抑制する効果がある
optimizer = torch.optim.Adam(model.parameters(), lr=0.1, weight_decay=1e-4)

In [6]:
# ===============================
# GCN モデルの学習関数
# 引数 epoch：学習エポック数
# ===============================
def train(epoch):
    # モデルを訓練モードに設定（Dropout などが有効になる）
    model.train()

    # 学習開始時刻を記録（経過時間を計測するため）
    start = time.time()

    # 指定されたエポック数だけ学習を繰り返す
    for epoch in range(epoch):
        # 勾配の初期化（前エポックの値をクリア）
        optimizer.zero_grad()

        # 順伝播を実行して出力（全ノードに対してクラスごとの対数確率を出力）
        out = model(data)

        # 損失関数の計算（教師あり損失）
        # - 対象：train_mask が True のノードだけ
        # - 損失関数：負の対数尤度損失（log_softmax に対応）
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])

        # 誤差逆伝播によって勾配を計算
        loss.backward()

        # 勾配に基づいてパラメータを更新
        optimizer.step()

        # テストマスク上でモデルの分類精度（accuracy）を計算
        acc = calc_acc(model)

        # 経過時間を記録（1エポック目からの合計）
        total_time = time.time() - start

        # エポック番号・学習時間・精度を表示
        print(
            str(epoch + 1) + " エポック目",
            format(total_time, ".2f") + " 秒",
            "精度 " + format(acc, ".4f"),
        )

In [7]:
# train(30)

In [8]:
# ===============================
# GCNモデルの定義と最適化手法（SGD）の設定
# ===============================

# GCNモデルのインスタンス化
# - 入力次元：dataset.num_node_features → Reddit では 602次元
# - 中間層次元：32次元（任意に設定）
# - 出力次元：dataset.num_classes → Reddit では 41クラス（subredditカテゴリ）
model = GCN(dataset.num_node_features, 32, dataset.num_classes)

# 最適化手法に SGD（確率的勾配降下法）を使用
# - 学習率 lr=0.1：1ステップの更新量（比較的大きめ）
# - weight_decay=1e-4：L2正則化係数（過学習防止）
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=1e-4)

In [9]:
# ===============================
# GCNの近傍サンプリング付きミニバッチ学習
# - NeighborLoader による大規模グラフ学習の効率化
# ===============================
def train_neighborhood_sampling(epoch, batch_size=128):
    # モデルを訓練モードに設定（Dropoutなど有効化）
    model.train()

    # NeighborLoader を用いてサンプルバッチを構築
    loader = NeighborLoader(
        data,  # グラフ全体のデータオブジェクト
        num_neighbors=[5]
        * 2,  # 各 GCN層で 5-hop までの隣接ノードをサンプリング（2層分）
        batch_size=batch_size,  # 1回のバッチに含める中心ノード数
        input_nodes=data.train_mask,  # バッチの中心ノード候補（訓練ノードのみ）
    )

    # 時間計測の開始
    start = time.time()

    # エポック数（epoch）だけ学習ループを回す
    for epoch in range(epoch):
        for sampled_data in loader:
            # 勾配初期化
            optimizer.zero_grad()

            # サンプリングされたサブグラフをモデルに入力して予測出力を得る
            out = model(sampled_data)

            # ロス計算（バッチサイズ分の中心ノードに対して）
            # - NeighborLoader によるサンプルは「中心ノード + その近傍」で構成される
            # - `out[:batch_size]` により中心ノード部分のみを損失計算の対象とする
            loss = F.nll_loss(out[:batch_size], sampled_data.y[:batch_size])

            # 勾配計算と最適化ステップ
            loss.backward()
            optimizer.step()

        # 1エポックごとに全体精度を評価（グローバルテストマスクを使用）
        acc = calc_acc(model)

        # 経過時間を計算
        total_time = time.time() - start

        # エポックごとの結果を表示
        print(
            str(epoch + 1) + " エポック目",
            format(total_time, ".2f") + " 秒",
            "精度 " + format(acc, ".4f"),
        )

In [ ]:
# NeighborLoader による近傍サンプリングを使うことで、
# - 各エポックで処理するノード数が少なくなるため、1エポックあたりの計算時間が短縮される
# - また、ミニバッチ学習のため、1エポック内に複数回パラメータが更新される（バッチ数ぶん）
# → 結果として、全体の学習が高速に進み、必要なエポック数も少なくて済む傾向がある
train_neighborhood_sampling(3)